In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, UnexpectedAlertPresentException, NoAlertPresentException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import os
from datetime import datetime
from dotenv import load_dotenv
from pathlib import Path
from selenium.webdriver.chrome.options import Options

In [23]:
# .env 파일 로드
load_dotenv()

DOWNLOAD_TIMEOUT = 60  # 다운로드 대기 시간

# Chrome 옵션 설정
options = webdriver.ChromeOptions()
download_prefs = {
    # "download.default_directory": os.path.expanduser(os.getenv('DOWNLOAD_PATH')), 이거 안됨 expanduser
    "download.default_directory": os.path.abspath(os.getenv('DOWNLOAD_PATH')),
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", download_prefs)

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 30)

try:
    # 페이지 열기 및 로그인
    driver.get(os.getenv('BASE_URL'))
    time.sleep(2)

    # 로그인 정보 입력
    userid_field = driver.find_element(By.ID, "userid")
    password_field = driver.find_element(By.ID, "passwd")
    
    userid_field.clear()
    password_field.clear()
    
    userid_field.send_keys(os.getenv("SELLMATE_ID"))
    password_field.send_keys(os.getenv("SELLMATE_PW"))
    
    driver.find_element(By.CLASS_NAME, "login_btn").click()
    time.sleep(2)

    # 재고 보고서 페이지 접근
    driver.get(f"{os.getenv('BASE_URL')}/stock/progressRpt.asp")
    time.sleep(2)

    driver.find_element(By.ID, "date_today").click()
    driver.find_element(By.ID, "search-btn").click()
    time.sleep(2)

    excel_dropdown = driver.find_element(By.ID, "excel_dropdown")
    excel_dropdown.click()
    time.sleep(1)

    main_window = driver.current_window_handle  # 현재 창 핸들 저장
    
    driver.find_element(By.XPATH, "//a[contains(text(), 'XLS양식설정')]").click()

    for handle in driver.window_handles:
        if handle != main_window:
            driver.switch_to.window(handle)
            time.sleep(3)  # 잠시 대기
            
            # Alert이 있는 경우 처리
            try:
                while True:
                    try:
                        alert = driver.switch_to.alert
                        alert.accept()  # Alert 확인 클릭
                        time.sleep(1)  # 잠시 대기
                    except :
                        break
            except Exception as e:  # 모든 예외를 포착
                if isinstance(e, UnexpectedAlertPresentException): # UnexpectedAlertPresentException 처리 코드
                    print("예기치 않은 Alert가 발생했습니다.")
                    raise e
                
                elif isinstance(e, NoAlertPresentException): # NoAlertPresentException 처리 코드
                    print("Alert가 없습니다.")
                    raise e
                    
                else: # 다른 예외 처리
                    print(f"처리되지 않은 예외 발생: {e}")
                    raise e

            # "db_update" 선택
            template_select = driver.find_element(By.CLASS_NAME, "xls-template-options")
            for option in template_select.find_elements(By.TAG_NAME, "option"):
                if option.text == "db_update":
                    option.click()
                    break
            
            time.sleep(1)
            driver.close()
            break

    driver.switch_to.window(main_window)  # 메인 창으로 돌아가기

    excel_dropdown.click()
    time.sleep(1)

    # '엑셀다운로드' 항목 선택
    excel_download_option = wait.until(
        EC.element_to_be_clickable((By.XPATH, "//li[@role='presentation' and @onclick='popXls();']/a"))
    )
    excel_download_option.click()

    excel_file_name = ""

    # 새 창에서 다운로드 버튼 클릭
    for handle in driver.window_handles:
        if handle != main_window:
            driver.switch_to.window(handle)
            try:
                # 다운로드 버튼이 있는 테이블 행 추출
                row = wait.until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, "table.origin_table tbody:first-of-type tr:first-of-type")
                    )
                )

                # 다운로드 버튼 클릭
                download_button = WebDriverWait(row, 30).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "td:last-child button.btn-success"))
                )

                # 완료시간 추출
                completed_time_cell = row.find_element(By.CSS_SELECTOR, "td:nth-child(5)")
                completed_time = datetime.strptime(completed_time_cell.text, "%Y-%m-%d %H:%M:%S")
                excel_file_name = f"stk_optOrdProgress_{completed_time.strftime('%Y%m%d_%H%M%S')}.csv"

                # 다운로드 버튼 클릭
                download_button.click()

                # 다운로드 완료 대기 개선
                downloaded_file_path = os.path.join(os.getenv('DOWNLOAD_PATH'), excel_file_name)
                download_path = Path(downloaded_file_path)
                wait_start = time.time()
                
                while not download_path.exists() or download_path.stat().st_size == 0:
                    if time.time() - wait_start > DOWNLOAD_TIMEOUT:
                        raise TimeoutException(f"파일 다운로드가 {DOWNLOAD_TIMEOUT}초 내에 완료되지 않았습니다.")
                    time.sleep(0.5)
                print(f"다운로드 완료: {downloaded_file_path}")

            except TimeoutException as e:
                print(f"오류 발생: {e}")
                raise e
            
            driver.close()
            break

    # CSV 파일 처리
    time.sleep(5)
    csv_path = os.path.join(os.getenv('DOWNLOAD_PATH'), excel_file_name)

    # 파일 존재 확인
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"CSV 파일을 찾을 수 없습니다: {csv_path}")
    
    try:
        index_df = pd.read_csv(csv_path, encoding='CP949')

    except Exception as e:
        print(f"CSV 파일 처리 중 오류 발생: {e}")
        raise  # 에러 전파하여 디버깅 용이하게

finally:
    # 브라우저 닫기
    driver.quit()

다운로드 완료: C:/Users/hyunb/Downloads\stk_optOrdProgress_20250116_154152.csv


In [24]:
index_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9427 entries, 0 to 9426
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   바코드번호       9427 non-null   object 
 1   공급처코드       9427 non-null   int64  
 2   상품명         9427 non-null   object 
 3   옵션내용        9427 non-null   object 
 4   현재고         9427 non-null   int64  
 5   20250116    9427 non-null   int64  
 6   Unnamed: 6  0 non-null      float64
dtypes: float64(1), int64(3), object(3)
memory usage: 515.7+ KB


In [26]:
from supabase import create_client
import os
from dotenv import load_dotenv
from datetime import datetime

# 환경 변수 로드
load_dotenv()

# Supabase 클라이언트 초기화
supabase = create_client(
    os.getenv('SUPABASE_URL'),
    os.getenv('SUPABASE_KEY')
)

try:
    # 필요한 컬럼만 선택
    df_for_product = index_df[['바코드번호', '공급처코드', '상품명', '옵션내용']]
    df_for_product.columns = ['barcode', 'supplier_id', 'name', 'option_name']
    df_for_inventory = index_df[['바코드번호', '현재고']]
    df_for_inventory.columns = ['product_barcode', 'current_stock']
    
    # DataFrame을 딕셔너리 리스트로 변환
    product_records = df_for_product.to_dict('records')
    inventory_records = df_for_inventory.to_dict('records')

    supabase.table('products').upsert(
        product_records,
        on_conflict='barcode'
    ).execute()

    supabase.table('product_inventory_records').insert(
        inventory_records
    ).execute()
    
except Exception as e:
    print(f"Error processing barcode {row['바코드번호']}: {str(e)}")

print("데이터베이스 업데이트가 완료되었습니다.")

데이터베이스 업데이트가 완료되었습니다.


현재 필요한 작업

1. products에 upsert할 때, 기존 products와 비교하여 is_deleted = true로 업데이트

2. product_inventory_records에 insert 할 때, 기존 product_inventory_records와 비교하여 insert
    조건
    1) 기존에 없던 product이다.
    2) 최신의 current_stock과 값이 다르다.

3. product_lead_times product_inventory_records에 insert한 product에 한해 insert한다.
    product_lead_times.days의 값은 다음과 같다.
    1) default
      - case(한국)


